## Stock Sentiment Analysis using News Headlines

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('NEWS_YAHOO_stock_prediction.csv', encoding = "ISO-8859-1")

In [3]:
# Convert 'Date' column to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Select 'title' and 'content' columns as input
inputs = df[['title', 'content']]
inputs

C:\Users\PC-DELL\AppData\Local\Temp\ipykernel_9140\3587074723.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


,title,content
Date,,
2020-01-27,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...
2020-01-27,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...
2020-01-27,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...
2020-01-27,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...
2020-01-27,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...
...,...,...
2012-07-31,What s Driving China s Real Estate Rally Par...,In the preceding posts I examined the first t...
2012-07-31,50 Stocks With The Best Dividend Growth In Jul...,Here is a current sheet of companies with fast...
2012-07-23,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...


In [4]:
from sklearn.model_selection import train_test_split
train_size = 0.8  # 80% for training, 20% for testing
train, test = train_test_split(inputs, train_size=train_size, shuffle=False)

# Example usage:
print("Train inputs shape:", train.shape)
print("Test inputs shape:", test.shape)
train

Train inputs shape: (12772, 2)
Test inputs shape: (3194, 2)


,title,content
Date,,
2020-01-27,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...
2020-01-27,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...
2020-01-27,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...
2020-01-27,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...
2020-01-27,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...
...,...,...
2016-06-06,Western Union Expands Digital Business Retain...,On Jun 7 2016 we updated our research report...
2016-06-06,The Zacks Analyst Blog Highlights Microsoft ...,For Immediate Release\n\n\tChicago IL June ...
2016-06-06,Dycom Industries Movado Group Church Dwigh...,For Immediate Release\n\n\tChicago IL June ...


In [5]:
data = train.iloc[:, 0:2]  # Selecting columns 2 to 26 from the training set (assuming 'title' and 'content' are in these columns)
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)
# Combine 'title' and 'content' columns into a single string per row
data['combined'] = data.apply(lambda row: ' '.join([str(row['title']), str(row['content'])]), axis=1)

# Dropping the original 'title' and 'content' columns as they are now combined
data.drop(['title', 'content'], axis=1, inplace=True)
data
# Display the first 5 rows of the modified DataFrame
print(data.head(1))

                                                     combined
Date                                                         
2020-01-27  Apple Set To Beat Q  Earnings Estimates  Tech ...


In [6]:
# Convertng headlines to lower case
for index in data:
    data[index]=data[index].str.lower()
data

,combined
Date,
2020-01-27,apple set to beat q earnings estimates tech ...
2020-01-27,tech daily intel results netflix surge appl...
2020-01-27,monster stock market predictions for the wee...
2020-01-27,apple earnings preview g launch expanding s...
2020-01-27,buy surging apple microsoft stock before qua...
...,...
2016-06-06,western union expands digital business retain...
2016-06-06,the zacks analyst blog highlights microsoft ...
2016-06-06,dycom industries movado group church dwigh...


                                                   combined sentiment
Date                                                                 
2020-01-27   apple set to beat q earnings estimates tech...  positive
2020-01-27   tech daily intel results netflix surge appl...   neutral
2020-01-27  monster stock market predictions for the wee...   neutral


In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Ensure the model is in evaluation mode
model.eval()

c:\Users\PC-DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [8]:
import pandas as pd
from transformers import 













, RobertaForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Ensure the model is in evaluation mode
model.eval()

# Function to process a batch of data
def process_batch(text_batch):
    # Tokenize and encode the text data
    encoded_inputs = tokenizer.batch_encode_plus(
        text_batch,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    
    # Classify the sentiments
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        logits = outputs.logits
    
    # Convert logits to probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get the sentiment predictions
    sentiment_labels = torch.argmax(probs, dim=1).tolist()
    
    # Map the sentiment predictions to labels (0: negative, 1: neutral, 2: positive)
    sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    sentiment_results = [sentiment_map[label] for label in sentiment_labels]
    
    return sentiment_results

In [68]:
batch_size = 1000  # Define your batch size
num_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)
sentiments = []

for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(data))
    text_batch = data['combined'].iloc[batch_start:batch_end].tolist()
    batch_sentiments = process_batch(text_batch)
    sentiments.extend(batch_sentiments)

# Add the sentiment results to the DataFrame
data['sentiment'] = sentiments

# Save the results to a new CSV file
data.to_csv('Apple Dataset corrected.csv')  # replace with the desired output path